# Setup Variables and home directory

MNIST, FashionMNIST, GTSRB, Cifar10

In [1]:
# set homw directory
import os
from pathlib import Path

base = Path().cwd()

if base.name != 'runtime-monitoring':
    os.chdir('../')
    base = Path().cwd()

# Libraries

In [2]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import json
import time
from fastprogress import progress_bar

In [3]:
import torch
import torchvision
import torch.nn as nn
import torch.backends.cudnn as cudnn
import torch.nn.functional as F
from torchinfo import summary

cudnn.benchmark = True
torch.set_float32_matmul_precision('high')

In [4]:
from utilities.utils import *
from utilities.scaleFunctions import *
from utilities.pcaFunctions import *
from utilities.pathManager import fetchPaths
from utilities.MonitorBDD import MonitorBDD

In [5]:
# disable warnings
import warnings
warnings.filterwarnings('ignore')

### Setup Variables

In [6]:
SEED = 42
CUDA = 0
GPU_NAME = f'cuda:{CUDA}'

np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)

device = get_device(GPU_NAME)

# Paths

In [7]:
# DATASET = 'MNIST'
# DATASET = 'FashionMNIST'
DATASET = 'GTSRB'

LHLs = {
    'MNIST': 60,
    'FashionMNIST': 60,
    'GTSRB': 50
}


LHL = LHLs[DATASET]

In [8]:
paths = fetchPaths(base, DATASET, '', False)

path_data = paths['data']
path_lhl = paths['lhl']

path_saved_model = paths['saved_models'].parent

configs = load_json(paths['configuration'])
config = configs['configuration']
model_setup = configs['model_setup']
model_config = configs['model_config']
optim_name = list(config['optimizer'].keys())[0]
optim_args = config['optimizer'][optim_name]
scheduler_name = list(config['scheduler'].keys())[0]
scheduler_args = config['scheduler'][scheduler_name]
batch_size = model_config['batch_size']

In [9]:
POSTFIX = f'{optim_name}-{batch_size}-{LHL}'
POSTFIX += '_selected_classes'
FILENAME_POSTFIX = f'{DATASET}_{POSTFIX}'

FILENAME_POSTFIX

'GTSRB_AdamW-32-50_selected_classes'

In [10]:
saved_model = list(path_saved_model.glob(f"*/{FILENAME_POSTFIX}*.pth.tar"))

if len(saved_model) > 0: saved_model = saved_model[0]
else: saved_model = None

# Import Data

In [12]:
from models.mnist_model import MNIST_Model
from models.fashionmnist_model import FashionMNIST_CNN
from models.gtsrb_model import GTSRB_CNN

from models.transform import transform

models = {
    'mnist': MNIST_Model,
    'fashionmnist': FashionMNIST_CNN,
    'gtsrb': GTSRB_CNN
}

model_ = models[DATASET.lower()]
transform_ = transform[DATASET.lower()]

In [13]:
feature_names = get_labels(DATASET)

train_data = get_dataset(DATASET, path_data, train=True, transform=transform_['train'])
test_data = get_dataset(DATASET, path_data, train=False, transform=transform_['test'])
len(train_data), len(test_data)

(64000, 4224)

In [14]:
skip_classes = [2, 5, 35, 40]

new_classes = np.setdiff1d( np.array(list(feature_names.keys())), skip_classes )

class_map = { c:i for i, c in enumerate(new_classes)}
class_map_rev = {v:k for k,v in class_map.items()}


def collate_fn(batch):
    id_drop = []
    for i, (_, y) in enumerate(batch):
        if y in skip_classes:
            id_drop.append(i)
    
    for i in id_drop[::-1]:
        batch.pop(i)
    
    x = torch.stack([x for x,_ in batch])
    y = torch.tensor([y for _,y in batch])
    
    y.apply_(lambda c: class_map[c])
    
    return x, y

In [15]:
trainloader = get_dataLoader(train_data, model_config['batch_size'], True, collate_fn=collate_fn)
testloader = get_dataLoader(test_data, model_config['batch_size'], False, collate_fn=collate_fn)

In [16]:
def create_model(lhl, num_classes):
    
    # model
    model_setup['last_hidden_neurons'] = lhl
    
    # torch 2.0 compile and parallel data training
    model = model_(**model_setup, outneurons = num_classes).to(device)
    model = torch.compile(model)
    nn.DataParallel(model, device_ids=[CUDA])
    
    # loss function
    loss_function = nn.CrossEntropyLoss()
    
    # optimizer and scheduler
    optimizer = getattr(torch.optim, optim_name)(model.parameters(), lr=model_config['lr'], **optim_args)
    scheduler = getattr(torch.optim.lr_scheduler, scheduler_name)(optimizer, **scheduler_args)
        
    return model, loss_function, optimizer, scheduler

In [17]:
if DATASET=='MNIST': model_config['epochs'] = 3
if DATASET=='GTSRB': model_config['epochs'] = 8

In [18]:
def start_training_testing(model_name, model, loss_function, optimizer, scheduler):
    # training testing attributes
    kwargs = {
        'model': model,
        'loss_function': loss_function,
        'optimizer': optimizer,
        'lr_scheduler': scheduler,
        'device': device,
        'model_path': path_saved_model / f"{model_name}.pth.tar",
        'trainloader': trainloader,
        'testloader': testloader,
        'config': model_config
    }

    # run training testing
    return run_training_testing(**kwargs)

In [19]:
model, loss_function, optimizer, scheduler = create_model(LHL, new_classes.shape[0])

if saved_model is None:
    (train_losses,
    train_accs,
    test_losses,
    test_accs,
    train_loss,
    train_acc,
    test_loss,
    test_acc,
    confusion_matrix_train,
    confusion_matrix_test,
    saved_model) = start_training_testing(FILENAME_POSTFIX, model, loss_function, optimizer, scheduler)

In [20]:
load_checkpoint(model, saved_model)

In [21]:
trainloader = get_dataLoader(train_data, model_config['batch_size'], True)
testloader = get_dataLoader(test_data, model_config['batch_size'], False)

In [22]:
def get_logits(loader):
    df = pd.DataFrame()

    with torch.no_grad():

        for x, y in progress_bar(loader):

            # extract last hidden layer and predicted class
            logits, y_pred = model.output_last_layer(x.to(device))
            
            y_pred = y_pred.argmax(dim=1).cpu()

            temp_df = pd.DataFrame(logits.cpu().numpy())
            temp_df['y'] = y.cpu().numpy()
            temp_df['true'] = temp_df['y'] == y_pred.apply_(lambda c: class_map_rev[c]).numpy()

            df = pd.concat([df, temp_df])

    return df

In [23]:
df_train = get_logits(trainloader)

In [24]:
df_test = get_logits(testloader)

In [61]:
df_train.shape, df_test.shape

((64000, 52), (4224, 52))

In [25]:
# select all true classified
df_true = pd.concat([df_train[df_train["true"] == True].copy(), df_test[df_test["true"] == True].copy()])
df_true = df_true.drop("true", axis=1).reset_index(drop=True)

# select only true classified
# df_true = df_train[df_train["true"] == True].copy()
# df_true = df_true.drop("true", axis=1).reset_index(drop=True)

df_true.shape

(58979, 51)

In [26]:
# fit scaler and pca
scaler_ = fitStandardScalerSingle(df_true, LHL)
pca_ = fitPCASingle(df_true, scaler=scaler_, numNeurons=LHL)

In [27]:
df_train_pca = applyPCASingle(df_train, scaler_, pca_, numNeurons=LHL)
df_test_pca = applyPCASingle(df_test, scaler_, pca_, numNeurons=LHL)
df_true_pca = applyPCASingle(df_true, scaler_, pca_, numNeurons=LHL)

In [28]:
gte_mean, top_third = neuronsLoadingsSingle(pca_, numNeurons=LHL, var_thld=0.9, loadings_thld=0.5)

# Build BDD

In [31]:
flavor = 'raw'

MonitorConfig = {
    'threshold': 'Median', # ReLU, Mean, Median, 25%-Q, 75%-Q
    'neuron_selection': 'GTE Mean', # None, GTE Mean, Top Third
    'eta': 0
}

In [32]:
# translate MonitorConfig to numbers
thld_p = {'Median': 0.5, '25%-Q': 0.25, '75%-Q': 0.75, 'ReLU': 0, 'Mean': 1}
thld_p = thld_p[MonitorConfig['threshold']]

eta = MonitorConfig['eta']

neurons = {'None': [], 'GTE Mean': gte_mean, 'Top Third': top_third}
neurons = neurons[MonitorConfig['neuron_selection']]

if flavor=='pca' and neurons!=[]:
    nerons = [f'x{i}' for i in range(numComponents(pca_))]

# calculate threshold
if thld_p == 1:
    thld = np.mean( df_true.drop('y', axis=1), axis=0)
if thld_p == 0:
    thld = np.zeros(df_true.drop('y', axis=1).shape[1])
else:
    thld = np.quantile( df_true.drop('y', axis=1), thld_p, axis=0)

In [33]:
# stop timer
st = time.perf_counter()

df_train_cls = df_train.loc[~df_train['y'].isin(skip_classes)].copy()
df_test_cls = df_test.loc[~df_test['y'].isin(skip_classes)].copy()
df_eval = pd.concat([df_train.loc[df_train['y'].isin(skip_classes)].copy(),
                     df_test.loc[df_test['y'].isin(skip_classes)].copy()])

# construct Monitor
patterns = MonitorBDD( df_true.shape[1]-1, thld, neurons=neurons, reorder=False, memory=10)

# build + evaluate
df_train_cls_copy, df_test_cls_copy, df_eval = patterns.add_dataframe( df_true, eta, eval_dfs=[
    df_train_cls, df_test_cls, df_eval] )

# stop timer
en = round(int(time.perf_counter() - st) / 60, 3)

# score
df_train_scores = patterns.score_dataframe_multi_eta(df_train_cls_copy, eta)
df_test_scores = patterns.score_dataframe_multi_eta(df_test_cls_copy, eta)
df_eval_scores = patterns.score_dataframe_multi_eta(df_eval, eta)

# add metadata
patterns.stats['num_observations'] = df_true.shape[0]

en

0.033

In [34]:
patterns.stats

,thld,eta,build_time_min,size_mb,reorder_time_min,num_patterns,num_unique_patterns_%,num_reorder,num_neurons,start_time,end_time,num_observations
1,NaN,0.0,0.0,146.5,0.0,58979.0,49.2,0.0,22.0,2023-11-14 12:59:50,2023-11-14 12:59:50,58979


In [35]:
df_train_scores.loc[ df_train_scores['y'] == 'all', df_train_scores.columns[:-3]]

,y,total_count,total_misclassified,unrecognized,unrecognized_and_misclassified,unrecognized_and_classified,NPR,NPV,specificity
39,all,58140.0,2805.0,2183.0,2183.0,0.0,0.037547,1.0,0.778253


In [36]:
df_test_scores.loc[ df_test_scores['y'] == 'all', df_test_scores.columns[:-3]]

,y,total_count,total_misclassified,unrecognized,unrecognized_and_misclassified,unrecognized_and_classified,NPR,NPV,specificity
39,all,3853.0,209.0,139.0,139.0,0.0,0.036076,1.0,0.665072


In [37]:
df_eval_scores.loc[ df_eval_scores['y'] == 'all', ['total_count','unrecognized_and_classified','NPR']]

,total_count,unrecognized_and_classified,NPR
4,6231.0,0.0,0.731183
